In [3]:
import re

In [4]:
# Descrição informal da gramática
# O símbolo inicial é <program>
# Não inclui whitespaces

grammar01 = """

<program> = <variable> <constants>? <functions>? <expression> 

<name> = <letter>(<alphanumeric_>)+
<variable> = variable <name>
<constants> = constants <name> (, <name>)*
<functions> = functions <name> (, <name>)*

<decimal> = <alphanumeric>+ [\.<alphanumeric>+]
<expression> = <term> [(+|-) <expression>] 
<term> = <power> [* <term>]
<power> = <factor> [^ <power>]
<factor> =  <decimal> | <name> | (<expression>)

"""

In [5]:
example01 = """

variable x
constants A, B, C
functions f
expression: (A + x + f)'

"""

example02 = """

variable t
constants k
functions x, y
expression: k*(x + y)*(x - y)

"""

example03 = """

variable t
constants k
functions x, y
expression: (x + t)*x + 2*t

"""

example04 = """

variable t2
constants A
functions x, y, z
expression: (x+Ay)*(x^2-A-z+t2)'

"""

In [6]:
example = example04

In [7]:
# Obs: name = [a-zA-Z][a-zA-Z0-9]*
header_regex = r'''\s*variable\s([a-zA-Z][a-zA-Z0-9]*)
(?:\s*constants\s+([a-zA-Z][a-zA-Z0-9]*(?:\s*,\s*[a-zA-Z][a-zA-Z0-9]*)*))?
(?:\s*functions\s+([a-zA-Z][a-zA-Z0-9]*(?:\s*,\s*[a-zA-Z][a-zA-Z0-9]*)*))?(.+)'''

m = re.match(header_regex,example,re.S)
variable, constants, functions, expression = m.groups()

In [ ]:
print(variable)
print(constants)
print(functions)
print(variable)

t2
A
x, y, z
t2


In [156]:
class Node:
    """
    Representa uma operação com número arbitrário de filhos ou um fator.
    """
    def __init__(self, name, children=None):
        self.children=children
        self.name=name

    def print(self):
        a = self.name
        if self.children is not None:
          a += '('
          a += self.children[0].print()
          for c in range(1, len(self.children)):
            a += ', ' + self.children[c].print()
          a += ')'
        return a

class Parser:
    """
    Realiza o parsing.
    """
    def __init__(self, s):
        # lex é um generator que retorna o próximo token encontrado, podendo ser ele:
        # [0-9]+(\.[0-9]+)?        Floats, ex: "4", "5.3", "22.75"
        # [a-zA-Z]([_a-zA-Z0-9])*  Nomes de funções ou variáveis compostos por 
        #         um caractere seguido por 0 ou mais caracteres ou underscores
        # \(|\)|\+|\-|\*           Qualquer outro delimitador válido na linguagem
        self.lex = re.finditer(r'\s*([0-9]+(\.[0-9]+)?|[a-zA-Z]([_a-zA-Z0-9])*|\(|\)|\+|\-|\*|\^)', s)
        self.current = self.next()

    def next(self):
      try:
        match = next(self.lex)
      except StopIteration:
        return '\0'
      return match.group().strip()

    def parse(self):
        return self.Expression()

    def Expression(self):
        l = self.Term()
        if self.current == '+' or self.current == '-':
          op = self.current
          self.current = self.next()
          r = self.Expression()
          if r != None:
              return Node(op, [l, r])
          return None
        return l

    def Term(self):
        l = self.Power()
        if self.current == '*':
            self.current = self.next()
            r = self.Term()
            if r == None:
                return None
            return Node("*", [l, r])
        return l
    
    def Power(self):
      l = self.Factor()
      if self.current == '^':
          self.current = self.next()
          r = self.Power()
          if r == None:
              return None
          return Node("^", [l, r])
      return l


    def Factor(self):
        if self.current == '(':
            self.current = self.next()
            r = self.Expression()
            if self.current == ')':
                self.current = self.next()
                return r
            return None
        l = self.current
        self.current = self.next()
        
        return Node(l)

In [157]:
# Operações básicas
tree = Parser('1*23').parse()
assert tree.print() == "*(1, 23)"

tree = Parser('25+   4.74').parse()
assert tree.print() == "+(25, 4.74)"

tree = Parser('  32 - 15 ').parse()
assert tree.print() == "-(32, 15)"

In [158]:
# Ordem das operações
tree = Parser('1*23+5').parse()
assert tree.print() == "+(*(1, 23), 5)"

tree = Parser('1 * (23+5)').parse()
assert tree.print() == "*(1, +(23, 5))"

In [161]:
# Exemplos complexos
tree = Parser('k*(x + y)*(x - y)').parse()
assert tree.print() == "*(k, *(+(x, y), -(x, y)))"

tree = Parser('(x + t)*x + 2*t').parse()
assert tree.print() == "+(*(+(x, t), x), *(2, t))"

tree = Parser('(x+Ay)*(x^2-A-z+t2)').parse()
assert tree.print() == "*(+(x, Ay), -(^(x, 2), -(A, +(z, t2))))"